In [2]:
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
import subprocess
import os
import time

# Start time
start_time = time.time()

# Input and output paths
shapefile_path = r'C:\projects\New_Script_SCP\outputs2\sample_6\sampl_6.shp'
dem_path = r'C:\projects\New_Script_SCP\outputs2\sample_6\Merged_DEM6.tif'
output_contour_path = r'C:\projects\New_Script_SCP\outputs2\sample_6_new\Contours.shp'
output_max_contour_path = r'C:\projects\New_Script_SCP\outputs2\sample_6_new\MaxContourPolygon2.shp'
output_centroid_path = r'C:\projects\New_Script_SCP\outputs2\sample_6_new\Centroid.shp'

# Load shapefile and compute centroid
gdf = gpd.read_file(shapefile_path)
gdf = gdf.to_crs(epsg=3857)  # Match CRS to the DEM
centroid = gdf.geometry.centroid.iloc[0]  # Assuming a single polygon

# Debug: Print centroid and save it as a shapefile
print(f"Centroid coordinates: {centroid}")
centroid_gdf = gpd.GeoDataFrame([{'geometry': centroid}], crs=gdf.crs)
centroid_gdf.to_file(output_centroid_path)
print(f"Centroid saved to {output_centroid_path}")

# Generate contours using gdal_contour
contour_interval = 0.1
try:
    result = subprocess.run([
        "gdal_contour",
        "-i", str(contour_interval),  # Contour interval
        "-a", "ELEV",  # Attribute name for elevation
        dem_path,
        output_contour_path
    ], check=True, capture_output=True, text=True)
    print(f"Contours generated and saved to {output_contour_path}")
except subprocess.CalledProcessError as e:
    print("Error generating contours:")
    print(e.stderr)  # Print error details from gdal_contour
    print(e.stdout)  # Optionally print standard output (if any)
    exit()

# Load contours and ensure they have a CRS
contours_gdf = gpd.read_file(output_contour_path)

# If CRS is None, reassign CRS from the DEM (assuming DEM is in EPSG:3857)
if contours_gdf.crs is None:
    contours_gdf.set_crs(epsg=3857, allow_override=True, inplace=True)

# Debug: Check contour attributes and CRS
print(f"Contour attributes: {contours_gdf.columns}")
print(f"Shapefile CRS: {gdf.crs}, Contours CRS: {contours_gdf.crs}")

# Buffer around centroid (optional, helps with small precision issues)
buffered_centroid = centroid.buffer(0.01)  # Adjust the buffer size if necessary

# Calculate distances from the centroid to each contour and add as a new column
contours_gdf['distance_to_centroid'] = contours_gdf.geometry.distance(centroid)

# Sort contours by distance to the centroid in ascending order
contours_gdf = contours_gdf.sort_values(by='distance_to_centroid')

# Initialize variables for the maximum contour
max_contour = None
max_elevation = float('-inf')

# Iterate over the sorted contours to find the maximum elevation contour containing the centroid
for _, row in contours_gdf.iterrows():
    # Convert geometry to polygon if it's a closed LineString
    polygon = convert_to_polygon(row['geometry'])
    if polygon and polygon.is_valid:
        # Check if the polygon contains the centroid
        if polygon.contains(centroid):
            print(f"Elevation: {row['ELEV']}, Contains centroid: True")
            if row['ELEV'] > max_elevation:
                max_contour = polygon
                max_elevation = row['ELEV']

# Debug: Output maximum contour information
if max_contour:
    print(f"Max elevation contour: {max_elevation}")
    max_contour_gdf = gpd.GeoDataFrame(
        [{'geometry': max_contour, 'elevation': max_elevation}],
        crs='EPSG:3857'
    )
    max_contour_gdf.to_file(output_max_contour_path)
    print(f"Maximum contour polygon saved at {output_max_contour_path}")
else:
    print("No valid contour found covering the centroid.")

# End time
end_time = time.time()

# Calculate and print the total time
total_time = end_time - start_time
print(f"Total execution time: {total_time:.2f} seconds")

Centroid coordinates: POINT (16451931.76034317 -3338950.438526478)
Centroid saved to C:\projects\New_Script_SCP\outputs2\sample_6_new\Centroid.shp
Contours generated and saved to C:\projects\New_Script_SCP\outputs2\sample_6_new\Contours.shp
Contour attributes: Index(['ID', 'ELEV', 'geometry'], dtype='object')
Shapefile CRS: EPSG:3857, Contours CRS: EPSG:3857
No valid contour found covering the centroid.
Total execution time: 0.38 seconds


In [3]:
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
import subprocess
import os
import time

# Start time
start_time = time.time()

# Input and output paths
shapefile_path = r'C:\projects\New_Script_SCP\outputs2\sample_4\sample_4.shp'
dem_path = r'C:\projects\New_Script_SCP\outputs2\sample_4\Merged_DEM3.tif'
output_contour_path = r'C:\projects\New_Script_SCP\outputs2\sample_4_new\Contours.shp'
output_max_contour_path = r'C:\projects\New_Script_SCP\outputs2\sample_4_new\MaxContourPolygon_new2.shp'
output_centroid_path = r'C:\projects\New_Script_SCP\outputs2\sample_4_new\Centroid.shp'

# Load shapefile and compute centroid
gdf = gpd.read_file(shapefile_path)
gdf = gdf.to_crs(epsg=3857)  # Match CRS to the DEM
centroid = gdf.geometry.centroid.iloc[0]  # Assuming a single polygon

# Debug: Print centroid and save it as a shapefile
print(f"Centroid coordinates: {centroid}")
centroid_gdf = gpd.GeoDataFrame([{'geometry': centroid}], crs=gdf.crs)
centroid_gdf.to_file(output_centroid_path)
print(f"Centroid saved to {output_centroid_path}")

# Generate contours using gdal_contour
contour_interval = 0.1
try:
    result = subprocess.run([
        "gdal_contour",
        "-i", str(contour_interval),  # Contour interval
        "-a", "ELEV",  # Attribute name for elevation
        dem_path,
        output_contour_path
    ], check=True, capture_output=True, text=True)
    print(f"Contours generated and saved to {output_contour_path}")
except subprocess.CalledProcessError as e:
    print("Error generating contours:")
    print(e.stderr)  # Print error details from gdal_contour
    print(e.stdout)  # Optionally print standard output (if any)
    exit()

# Load contours and ensure they have a CRS
contours_gdf = gpd.read_file(output_contour_path)

# If CRS is None, reassign CRS from the DEM (assuming DEM is in EPSG:3857)
if contours_gdf.crs is None:
    contours_gdf.set_crs(epsg=3857, allow_override=True, inplace=True)

# Debug: Check contour attributes and CRS
print(f"Contour attributes: {contours_gdf.columns}")
print(f"Shapefile CRS: {gdf.crs}, Contours CRS: {contours_gdf.crs}")

# Buffer around centroid (optional, helps with small precision issues)
buffered_centroid = centroid.buffer(0.01)  # Adjust the buffer size if necessary

# Initialize variables for the maximum contour
max_contour = None
max_elevation = float('-inf')

# Process each contour to convert closed LineStrings to Polygons
def convert_to_polygon(geometry):
    if isinstance(geometry, LineString) and geometry.is_ring:
        return Polygon(geometry)
    return None

contours_gdf['polygon'] = contours_gdf.geometry.apply(convert_to_polygon)

# Iterate over the contours to find the maximum elevation contour covering the centroid
for _, row in contours_gdf.iterrows():
    polygon = row['polygon']
    if polygon and polygon.is_valid:
        # Check if the polygon contains the centroid
        if polygon.contains(centroid):
            print(f"Elevation: {row['ELEV']}, Contains centroid: True")
            if row['ELEV'] > max_elevation:
                max_contour = polygon
                max_elevation = row['ELEV']

# Debug: Output maximum contour information
if max_contour:
    print(f"Max elevation contour: {max_elevation}")
    max_contour_gdf = gpd.GeoDataFrame(
        [{'geometry': max_contour, 'elevation': max_elevation}],
        crs='EPSG:3857'
    )
    max_contour_gdf.to_file(output_max_contour_path)
    print(f"Maximum contour polygon saved at {output_max_contour_path}")
else:
    print("No valid contour found covering the centroid.")
    
# End time
end_time = time.time()

# Calculate and print the total time
total_time = end_time - start_time
print(f"Total execution time: {total_time:.2f} seconds")

Centroid coordinates: POINT (16542558.80501196 -3229857.518137103)
Centroid saved to C:\projects\New_Script_SCP\outputs2\sample_4_new\Centroid.shp
Contours generated and saved to C:\projects\New_Script_SCP\outputs2\sample_4_new\Contours.shp
Contour attributes: Index(['ID', 'ELEV', 'geometry'], dtype='object')
Shapefile CRS: EPSG:3857, Contours CRS: EPSG:3857
Elevation: 213.4, Contains centroid: True
Elevation: 213.5, Contains centroid: True
Elevation: 213.20000000000002, Contains centroid: True
Elevation: 213.3, Contains centroid: True
Elevation: 212.9, Contains centroid: True
Elevation: 213.0, Contains centroid: True
Elevation: 213.10000000000002, Contains centroid: True
Elevation: 212.70000000000002, Contains centroid: True
Elevation: 212.8, Contains centroid: True
Elevation: 212.60000000000002, Contains centroid: True
Elevation: 212.4, Contains centroid: True
Elevation: 212.5, Contains centroid: True
Elevation: 212.3, Contains centroid: True
Elevation: 212.3, Contains centroid: True